In [1]:
import os
os.getcwd()
os.chdir('c:\\Users\\Stior\\Documents\\cours\\L3SID\\Projet-InterPromo\\ProjetInterpromo2020G8')

In [2]:
from pipeline import Pipeline, Process, Postprocess, Preprocess
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import PIL

In [3]:
class MyPreProcess(Preprocess):
    process_desc = "Exemple de pre-process -> ne fait rien"
    def run(self, images):
        pass


class MyProcess(Process):
    process_desc = "Exemple de process -> ne fait rien"
    def run(self, images):
        pass
    
    
class MyPostProcess(Postprocess):
    process_desc = "Exemple de post-process -> ne fait rien"
    def run(self, images):
        pass

In [4]:
pipeline = Pipeline()
process_array = np.array([MyPreProcess()
                          ,MyProcess()
                          ,MyPostProcess()
                          ,MyPreProcess(verbose=0)])
pipeline.add_processes(process_array)


Preprocess :  Exemple de pre-process -> ne fait rien
Process :  Exemple de process -> ne fait rien
Postprocess :  Exemple de post-process -> ne fait rien
Exemple de pre-process -> ne fait rien a été ajouté.
Exemple de process -> ne fait rien a été ajouté.
Exemple de post-process -> ne fait rien a été ajouté.
Exemple de pre-process -> ne fait rien a été ajouté.


In [5]:
pipeline.print_process()

Exemple de pre-process -> ne fait rien
Exemple de pre-process -> ne fait rien
Exemple de process -> ne fait rien
Exemple de post-process -> ne fait rien


In [6]:
img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 0)
img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 1)

#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A320_plane9.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A320_plane9.jpg', 1)


#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Air_Transat_Airbus_A330-300_V2_plane128.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Air_Transat_Airbus_A330-300_V2_plane128.jpg', 1)

#template = cv2.imread('./images/temp_normal_seat.jpg', 0)
#template = cv2.imread('./images/templateTest.jpg', 0)
#template = cv2.imread('./images/templateNormalSeat-SansBlanc.png', 0)
#template = cv2.imread('./images/templateNormalSeat-SansBlanc.jpg', 0)
template = cv2.imread('./images/templateTest-sansBlanc.png', 0)

#cv2.imshow('Test', img2)
#cv2.waitKey()
#cv2.destroyAllWindows() 

h, w = template.shape


In [7]:

methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

In [8]:

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.8
loc = np.where( res >= threshold)

coord = []

for pt in zip(*loc[::-1]):
    coord.append((pt[0], pt[1]))
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

for i in coord:
    cv2.circle(img_rgb, i, 1, (255, 0, 0), -1)

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows() 

print(coord)


with open("coord.txt", "a") as fichier:
    for xy in coord:
        fichier.write(str(xy[0]) + ", " + str(xy[1]) + "\n")



[(146, 517), (133, 537), (146, 537), (185, 537), (198, 537), (211, 537), (224, 537), (263, 537), (276, 537), (133, 557), (146, 557), (185, 557), (198, 557), (211, 557), (224, 557), (263, 557), (276, 557), (133, 577), (146, 577), (185, 577), (198, 577), (211, 577), (224, 577), (263, 577), (276, 577), (133, 597), (146, 597), (185, 597), (198, 597), (211, 597), (224, 597), (263, 597), (276, 597), (133, 617), (146, 617), (185, 617), (198, 617), (211, 617), (224, 617), (263, 617), (276, 617), (133, 637), (146, 637), (185, 637), (198, 637), (211, 637), (224, 637), (263, 637), (276, 637), (133, 657), (146, 657), (185, 657), (198, 657), (211, 657), (224, 657), (263, 657), (276, 657), (133, 677), (146, 677), (185, 677), (198, 677), (211, 677), (224, 677), (263, 677), (276, 677), (133, 697), (146, 697), (185, 697), (198, 697), (211, 697), (224, 697), (263, 697), (276, 697), (133, 717), (146, 717), (185, 717), (198, 717), (211, 717), (224, 717), (263, 717), (276, 717), (133, 737), (146, 737), (18

In [16]:
img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 0)
img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 1)
#template = cv2.imread('./images/temp_normal_seat.jpg', 0)
templates = ['./images/temp_business_seat_armrest_left.jpg', './images/temp_business_seat_armrest_right.jpg', './images/temp_business_elec1.jpg']
#templates = ['./images/templateBusinessElectricity.jpg']

In [10]:
for templ in templates:
    template = cv2.imread(templ, 0)
    h, w = template.shape
    
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    
    if templ == './images/temp_business_elec1.jpg':
        threshold = 0.7
    else:
        threshold = 0.7

    loc = np.where( res >= threshold)

    coord = []

    fichier = open("coord.txt", "a")

    if templ == './images/temp_business_elec1.jpg':
        fichier.write('-------------------\nPrise electrique \n')

    for pt in zip(*loc[::-1]):
        fichier.write(str(pt[0]) + ", " + str(pt[1]) + "\n")
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
    
    fichier.close()

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()
#cv2.imwrite('test.jpeg', img_rgb)

print(np.where( res >= threshold))

(array([334, 334, 334, 334, 334, 367, 367, 368, 399, 401, 431, 431, 435,
       435, 435], dtype=int64), array([140, 190, 217, 268, 269, 191, 218, 250, 190, 140, 191, 218, 149,
       250, 277], dtype=int64))


In [38]:
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-900_plane259.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-900_plane259.jpg', 1) #Ok
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-1000_plane258.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-1000_plane258.jpg', 1) #Ok
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A330_33E_plane255.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A330_33E_plane255.jpg', 1) #Ok
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/American_Airlines_777-300_ER_plane167.svg.png', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/American_Airlines_777-300_ER_plane167.svg.png', 1) #Null
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Asiana_Boeing_777-200_ER_D_plane206.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Asiana_Boeing_777-200_ER_D_plane206.jpg', 1) #Null - rectangle autour
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Avianca_Boeing_787-8_plane313.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Avianca_Boeing_787-8_plane313.jpg', 1) #Ok
img_gray = cv2.imread('./theo_remove_caracters_yellow.jpg', 0)
img_rgb = cv2.imread('./theo_remove_caracters_yellow.jpg', 1) #Ok


#template = cv2.imread('./images/temp_normal_seat.jpg', 0)
templates = ['./images/templateBusinessSeat1.jpg', 'templateTest.jpg', './images/templateNormalSeat-SansBlanc.jpg', './images/temp_normal_seat_elec.jpg']



In [40]:
#Rotation 45° gauche + noir => blanc

templateTest = PIL.Image.open(templates[0])
rotate = templateTest.rotate(45)
rotate = rotate.convert('RGBA')

data = np.array(rotate)   # "data" is a height x width x 4 numpy array
red, green, blue, alpha = data.T # Temporarily unpack the bands for readability


black_areas = (red == 0) & (blue == 0) & (green == 0)
data[..., :-1][black_areas.T] = (255, 255, 255) # Transpose back needed
rotate = PIL.Image.fromarray(data)
rotate.show()
rotate = rotate.convert('RGB')
rotate.save('templateTest.jpg')

In [41]:
templateTest = PIL.Image.open(templates[0])
rotate = templateTest.transpose(PIL.Image.FLIP_LEFT_RIGHT)

rotate.show()
rotate.save('templateTest.jpg')


#107-171-93

In [37]:
templateTest = PIL.Image.open(templates[2])
templateTest = templateTest.convert('RGBA')

data = np.array(templateTest)
red, green, blue, alpha = data.T

threshold = 200

white_areas = (red >= threshold) & (blue >= threshold) & (green >= threshold)
data[..., :-1][white_areas.T] = (255, 255, 255) # Transpose back needed
templateTest = PIL.Image.fromarray(data)
templateTest.show()



In [42]:
for templ in templates:
    template = cv2.imread(templ, 0)
    h, w = template.shape
    
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    
    threshold = 0.9
    if templ == 'templateTest.jpg':
        threshold = 0.60
    if templ == templates[0]:
        threshold = 0.60
    if templ == templates[3]:
        threshold = 0.7


    loc = np.where( res >= threshold)

    #fichier = open("coord.txt", "a")

    for pt in zip(*loc[::-1]):
        #fichier.write(str(pt[0]) + ", " + str(pt[1]) + "\n")
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
    
    #fichier.close()

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite('test.jpg', img_rgb)

#print(np.where( res >= threshold))

True

In [73]:
#Version 0

#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-900_plane259.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-900_plane259.jpg', 1) #Ok
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-1000_plane258.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-1000_plane258.jpg', 1) #Ok
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A330_33E_plane255.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A330_33E_plane255.jpg', 1) #Ok
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/American_Airlines_777-300_ER_plane167.svg.png', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/American_Airlines_777-300_ER_plane167.svg.png', 1) #Null
#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Asiana_Boeing_777-200_ER_D_plane206.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Asiana_Boeing_777-200_ER_D_plane206.jpg', 1) #Null - rectangle autour
img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Avianca_Boeing_787-8_plane313.jpg', 0)
img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Avianca_Boeing_787-8_plane313.jpg', 1) #Ok
#img_gray = cv2.imread('./theo_remove_caracters_yellow.jpg', 0)
#img_rgb = cv2.imread('./theo_remove_caracters_yellow.jpg', 1) #Ok


#template = cv2.imread('./images/temp_normal_seat.jpg', 0)
templates = ['./images/temp_business_turn_left.jpg', './images/temp_business_turn_right.jpg','./images/temp_normal_seat_elec.jpg']

for templ in templates:
    template = cv2.imread(templ, 0)
    h, w = template.shape
    
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    
    threshold = 0.9
    if templ == templates[0]:
        print('temp_business_turn_left')
        threshold = 0.60
    if templ == templates[1]:
        print('temp_business_turn_right')
        threshold = 0.60
    if templ == templates[2]:
        print('temp_normal_seat_elec')
        threshold = 0.65


    loc = np.where( res >= threshold)

    #fichier = open("coord.txt", "a")

    for pt in zip(*loc[::-1]):
        print(str(pt[0]) + ', ' + str(pt[1]) + '\n')
        #fichier.write(str(pt[0]) + ", " + str(pt[1]) + "\n")
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
    
    #fichier.close()

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite('test.jpg', img_rgb)

#print(np.where( res >= threshold))


temp_business_turn_left
135, 305

136, 305

216, 305

217, 305

135, 306

136, 306

216, 306

217, 306

135, 341

136, 341

216, 341

217, 341

135, 342

136, 342

217, 342

135, 376

216, 376

135, 377

136, 377

216, 377

217, 377

136, 378

217, 378

135, 412

136, 412

216, 412

217, 412

135, 413

136, 413

216, 413

217, 413

135, 533

136, 533

216, 533

217, 533

135, 534

136, 534

216, 534

217, 534

temp_business_turn_right
179, 305

259, 305

260, 305

178, 306

179, 306

259, 306

178, 341

179, 341

259, 341

260, 341

178, 342

179, 342

259, 342

179, 376

259, 376

260, 376

178, 377

179, 377

259, 377

260, 377

259, 378

179, 412

259, 412

260, 412

178, 413

179, 413

259, 413

179, 533

259, 533

260, 533

178, 534

179, 534

259, 534

temp_normal_seat_elec
190, 603

132, 604

133, 604

147, 604

162, 604

190, 604

204, 604

219, 604

246, 604

260, 604

275, 604

190, 624

204, 624

219, 624

133, 625

147, 625

162, 625

190, 625

204, 625

219, 625

246, 625


True